## Using SIMD operations and "rotate" gate to perform vector dot product

A powerful feature of some HE schemes is the ability to perform SIMD operations, doing the same calculation on multiple "slots" (i.e. elements of a vector).  The first part of a vector dot product - the component-wise multiplication - is therefore trivial.  However, we then need to sum over the elements to obtain the scalar product.  This can be done using ROTATE and ADD operations, as demonstrated in this notebook.

In [1]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

from pysheep import sheep_client

In [2]:
sheep_client.new_job()

{'content': '', 'status_code': 200}

In [4]:
sheep_client.set_context("SEAL")
sheep_client.set_input_type("int8_t")
sheep_client.get_nslots()

{'content': {'nslots': 2048}, 'status_code': 200}

With this set of parameters, we have 4 slots available, so can do a dot product of two 4-component vectors.
The circuit to perform this operation will be a MULTIPLY followed by a sequence of 3 ROTATEs and ADDs.

In [5]:
circuit = """
INPUTS input_0 input_1
CONST_INPUTS rotate_1
OUTPUTS output
input_0 input_1 MULTIPLY prod_r0
prod_r0 rotate_1 ROTATE prod_r1
prod_r0 prod_r1 ADD prod_s1
prod_r1 rotate_1 ROTATE prod_r2
prod_s1 prod_r2 ADD prod_s2
prod_r2 rotate_1 ROTATE prod_r3
prod_s2 prod_r3 ADD output
"""

In [6]:
sheep_client.set_circuit_text(circuit)
sheep_client.get_inputs()

{'content': ['input_0', 'input_1'], 'status_code': 200}

So the two input vectors are called input_0 and input_1. Let's assign them the values {1,2,3,4} and {5,6,7,8}

In [7]:
sheep_client.set_inputs({"input_0": [1,2,3,4], "input_1": [5,6,7,8]})

{'content': '', 'status_code': 200}

In [8]:
sheep_client.get_const_inputs()

{'content': ['rotate_1'], 'status_code': 200}

The circuit also takes a "const input" (that won't be encrypted) - this is how much we will ROTATE the vector by in each step, so just set it to -1

In [9]:
sheep_client.set_const_inputs({"rotate_1": -1})

{'content': '', 'status_code': 200}

In [10]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [11]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': True},
  'outputs': {'output': ['70,70,70,70']},
  'timings': {'decryption': '673.100000',
   'encryption': '1869.500000',
   'evaluation': '9188.300000',
   'output': '57.100000',
   'prod_r0': '117.700000',
   'prod_r1': '5285.300000',
   'prod_r2': '786.700000',
   'prod_r3': '92.900000',
   'prod_s1': '1624.800000',
   'prod_s2': '852.800000'}},
 'status_code': 200}

So, the output is '70,70,70,70' (we always get an output vector the same length as our input vector, even though in this case we only need one number), and 70 is indeed the scalar product of {1,2,3,4} and {5,6,7,8}.  


## Generalizing, and generating circuits

We probably don't want to write circuits by hand, particularly if we are dealing with long vectors (HElib and SEAL can, depending on parameter choices, offer thousands of slots).   We can write a simple python function to generate circuits for arbitrary length vectors:

In [12]:
def generate_vector_dot_product_circuit(input_0, input_1):
    """
    Given two input lists (must be equal in length) generate a SHEEP circuit to do the dot product
    """
    if len(input_0) != len(input_1):
        raise RuntimeError("input_0 and input_1 must be the same length")
    circuit_str = "INPUTS input_0 input_1\nCONST_INPUTS rotate_1\nOUTPUTS output prod_s1 prod_s2 prod_s3\ninput_0 input_1 MULTIPLY prod_r0\n"
    for i in range(len(input_0)-1):
        circuit_str += "prod_r{} rotate_1 ROTATE prod_r{}\n".format(i,i+1)
        if i==0:
            circuit_str += "prod_r0 prod_r1 ADD prod_s1\n"
        else:
            circuit_str += "prod_s{} prod_r{} ADD prod_s{}\n".format(i,i+1,i+1)
    circuit_str += "prod_s{} ALIAS output\n".format(i+1)
    return circuit_str

(Note that this function is also available in ```pysheep/mid_level_benchmarks.py```)

So, let's do a longer calculation in SEAL - multiply 2 vectors with 100 elements each, where each element is a random number between -10 and 10

In [28]:
import random
input_0 = []
input_1 = []
for i in range(100):
    input_0.append(random.randint(-10,10))
    input_1.append(random.randint(-10,10))

Lets quickly do the calculation in the clear so we know what answer to expect:

In [29]:
sum = 0
for i in range(len(input_0)):
    sum += input_0[i]*input_1[i]
sum

-54

In [30]:
sheep_client.new_job()

{'content': '', 'status_code': 200}

In [31]:
sheep_client.set_context("SEAL")

{'content': '', 'status_code': 200}

In [32]:
sheep_client.set_input_type("int16_t")

{'content': '', 'status_code': 200}

In [33]:
sheep_client.get_nslots()

{'content': {'nslots': 2048}, 'status_code': 200}

In [34]:
circuit = generate_vector_dot_product_circuit(input_0,input_1)
sheep_client.set_circuit_text(circuit)

{'content': '', 'status_code': 200}

In [35]:
sheep_client.set_inputs({"input_0":input_0, "input_1": input_1})

{'content': '', 'status_code': 200}

In [36]:
sheep_client.set_const_inputs({"rotate_1": -1})

{'content': '', 'status_code': 200}

In [37]:
sheep_client.set_timeout(60)

{'content': '', 'status_code': 200}

In [38]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [39]:
sheep_client.get_results()['content']

{'cleartext check': {'is_correct': True},
 'outputs': {'output': ['-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54,-54'],
  'prod_s1': ['-37,-19,8,19,26,33,-54,-30,65,7,-73,-101,24,56,-9,6,-9,-8,-8,-32,8,-2,-46,14,28,15,32,22,43,102,14,-4,-14,-70,-65,-3,56,-10,-28,-34,-35,-21,-16,-20,-14,15,-11,-12,22,4,-10,-32,-59,-27,18,36,6,51,126,56,-23,-65,-85,-92,-91,-15,44,24,-24,-12,-2,-35,-26,-13,62,84,15,4,6,-11,-18,20,0,66,96,60,63,81,0,-72,-70,-126,-14,-39,-86,25,84,75,27,-24'],
  'prod_s2': ['-49,1,7,46,32,-27,-24,5,37,-38,-129,-21,0,71,-18,6,-17,-8,-40,8,-34,-6,-28,24,33,42,27,70,97,62,50,-54,-34,-115,-23,11,32,-14,-58,-39,-51,-21,-36,-14,-5,-5,-3